In [1]:
from fastai.basics import *
from IPython.core.pylabtools import figsize
import gc, json
from pandas.io.json import json_normalize
from datetime import datetime
#import lightgbm as lgb
gc.enable()

/home/rubens/miniconda3/lib/python3.8/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


In [2]:
def load_tr(csv_path, nrows=None, skiprows=None):
    
    usecols = ['channelGrouping', 'date', 'device',
       'fullVisitorId', 'geoNetwork', 'totals',
       'trafficSource', 'visitId', 'visitNumber', 'visitStartTime' ]
    json_cols = ['device', 'geoNetwork', 'totals', 'trafficSource']
    ans = pd.DataFrame()
    trs = pd.read_csv(csv_path, 
            sep=',',
            usecols = usecols,
            converters={column: json.loads for column in json_cols}, 
            dtype={'fullVisitorId': 'str',
                  'channelGrouping': 'str',                 
                  'visitId':'int',
                  'visitNumber':'int',
                  'visitStartTime':'int'}, 
            parse_dates=['date'], 
            chunksize=100000,
            nrows=nrows,
            skiprows=skiprows)
    
    for tr in trs:
        tr.reset_index(drop=True, inplace=True)
        for column in json_cols:
            column_as_tr = json_normalize(tr[column])
            column_as_tr.columns = [f"{column}_{subcolumn}" for subcolumn in column_as_tr.columns]
            tr = tr.drop(column, axis=1).merge(column_as_tr, right_index=True, left_index=True)

        print(f"Loaded {os.path.basename(csv_path)}. Shape: {tr.shape}")
        tr_chunk = tr  #[features]
        del tr
        gc.collect()
        ans = pd.concat([ans, tr_chunk], axis=0).reset_index(drop=True)
        #print(ans.shape)
    return ans

In [6]:
PATH=Path('/home/rubens/googleanalytics/data')

tr = load_tr(PATH/'sample.txt')
print('train date:', min(tr['date']), 'to', max(tr['date']))

<ipython-input-2-198ed635a259>:25: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  column_as_tr = json_normalize(tr[column])


Loaded sample.txt. Shape: (9999, 56)
train date: 2016-09-02 00:00:00 to 2017-11-30 00:00:00


In [7]:
tr.columns

Index(['channelGrouping', 'date', 'fullVisitorId', 'visitId', 'visitNumber',
       'visitStartTime', 'device_browser', 'device_browserVersion',
       'device_browserSize', 'device_operatingSystem',
       'device_operatingSystemVersion', 'device_isMobile',
       'device_mobileDeviceBranding', 'device_mobileDeviceModel',
       'device_mobileInputSelector', 'device_mobileDeviceInfo',
       'device_mobileDeviceMarketingName', 'device_flashVersion',
       'device_language', 'device_screenColors', 'device_screenResolution',
       'device_deviceCategory', 'geoNetwork_continent',
       'geoNetwork_subContinent', 'geoNetwork_country', 'geoNetwork_region',
       'geoNetwork_metro', 'geoNetwork_city', 'geoNetwork_cityId',
       'geoNetwork_networkDomain', 'geoNetwork_latitude',
       'geoNetwork_longitude', 'geoNetwork_networkLocation', 'totals_visits',
       'totals_hits', 'totals_pageviews', 'totals_bounces', 'totals_newVisits',
       'totals_sessionQualityDim', 'totals_timeOnSite

In [8]:
#correct dtypes
tr["date"] = pd.to_datetime(tr["date"], infer_datetime_format=True, format="%Y%m%d")
tr['totals_hits'] = tr['totals_hits'].astype(float)
tr['totals_pageviews'] = tr['totals_pageviews'].astype(float)
tr['totals_timeOnSite'] = tr['totals_timeOnSite'].astype(float)
tr['totals_newVisits'] = tr['totals_newVisits'].astype(float)
tr['totals_transactions'] = tr['totals_transactions'].astype(float)
tr['device_isMobile'] = tr['device_isMobile'].astype(bool)
tr['trafficSource_isTrueDirect'] = tr['trafficSource_isTrueDirect'].astype(bool)


In [9]:
#replace all empty fields with NaN
Nulls = ['(not set)', 'not available in demo dataset', '(not provided)', 
         'unknown.unknown', '/', 'Not Socially Engaged']
for null in Nulls:    
    tr.replace(null, np.nan, inplace=True)

In [10]:
# target
tr['totals_totalTransactionRevenue'] = tr['totals_totalTransactionRevenue'].astype(float)
tr['totals_totalTransactionRevenue'].fillna(0, inplace=True)
target = tr['totals_totalTransactionRevenue']

In [16]:
tr.shape

(9999, 56)

In [17]:
tr.head(10).T

,0,1,2,3,4,5,6,7,8,9
channelGrouping,Organic Search,Referral,Direct,Organic Search,Organic Search,Referral,Referral,Organic Search,Organic Search,Organic Search
date,2017-10-16 00:00:00,2017-10-16 00:00:00,2017-10-16 00:00:00,2017-10-16 00:00:00,2017-10-16 00:00:00,2017-10-16 00:00:00,2017-10-16 00:00:00,2017-10-16 00:00:00,2017-10-16 00:00:00,2017-10-16 00:00:00
fullVisitorId,3162355547410993243,8934116514970143966,7992466427990357681,9075655783635761930,6960673291025684308,0166277907528479249,8349655975937271469,1332629902468998662,632878546807742341,1259490915281096752
visitId,1508198450,1508176307,1508201613,1508169851,1508190552,1508196701,1508152478,1508206208,1508207516,1508165159
visitNumber,1,6,1,1,1,1,1,1,1,2
visitStartTime,1508198450,1508176307,1508201613,1508169851,1508190552,1508196701,1508152478,1508206208,1508207516,1508165159
device_browser,Firefox,Chrome,Chrome,Chrome,Chrome,Chrome,Chrome,Chrome,Chrome,Safari
device_browserVersion,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
device_browserSize,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
device_operatingSystem,Windows,Chrome OS,Android,Windows,Windows,Macintosh,Macintosh,Windows,Macintosh,iOS


In [29]:
len(tf['fullVisitorId'].unique())

9209

In [35]:
lista_1 = tf['fullVisitorId'].tolist()
lista_1[:3]

['3162355547410993243', '8934116514970143966', '7992466427990357681']

In [41]:
from collections import Counter
lista_2 = [(k,v) for (k,v) in Counter(lista_1).items() if v > 1]
print(lista_2[:5])
print(len(lista_2))

[('8934116514970143966', 3), ('1259490915281096752', 2), ('6135613929977117121', 3), ('451521411412093630', 2), ('1172736694169070530', 2)]
636


In [43]:
### tinha 9999 ao inicio
### unicos 9209

agrupados = 0
for k,v in lista_2:
    agrupados = agrupados + v -1
agrupados    

790

In [ ]:
agr1 = tf['totals_visits'].groupby(tf['fullVisitorId']).size()


In [59]:
grouped =tf['totals_visits'].groupby(tf['fullVisitorId'])
grouped.agg('sum')

fullVisitorId
0000245437374675368    1
0001191766179392657    1
0003631840334189025    1
0004374401845204055    1
0004803397127474847    1
                      ..
9991882289486375263    1
9996018333110892344    1
9996276006553512804    1
99976789209401933      1
9999250019952621738    1
Name: totals_visits, Length: 9209, dtype: object

In [56]:
total = 0
for index, val in agr1.iteritems():
    if val > 1:
        print (index, val)
        total += (val -1)
print(total)        

0005096736076850458 2
0073418482524217161 2
011894735283045088 2
0140565629086135926 2
0149900644897532124 2
0150499846330462944 2
0158478110531564328 2
0185467632009737931 2
020651025263954888 2
0279576853267254132 2
0336854895417907444 2
0349004699334148471 2
041127596981749178 2
0432894311233772163 2
0436481626935381424 2
0440206142725306353 4
0444330656643598066 2
0458033026034702579 2
0463325773564352787 2
0474510610802118753 2
0493329590229002952 2
0514591268737702944 2
0536897889827593448 2
0569086010364471094 2
0571826267646040000 2
0600369081537544060 3
0646300986554067228 2
0671849830470961386 2
0688527857109237010 2
0705846205174200943 4
071727516194667087 2
0720794097063742520 2
0731685071486167443 2
0732747350124246982 2
0765488541386366209 3
0777417428217599872 2
0780206376162514125 2
0783993264243155236 2
0810134634005989709 2
0822148121562777607 2
0823099775394669327 3
0854783508496317255 2
0877259160433732747 2
0883102089127421220 2
0933552129991709243 2
09497189156434

In [45]:
tf.groupby('fullVisitorId').agg('totals_visits')

In [70]:
tf['totals_timeOnSite'].dropna().sum()

1326189.0

In [72]:
def my_agg(x):
    names = {
        'timeOnSite_sum':  x['totals_timeOnSite'].dropna().sum(),
        'timeOnSite_min':  x['totals_timeOnSite'].dropna().min(),
        'timeOnSite_max':  x['totals_timeOnSite'].dropna().max(),
        'timeOnSite_mean': x['totals_timeOnSite'].dropna().mean()
    }
    return pd.Series (names,index=[ 'timeOnSite_sum','timeOnSite_min','timeOnSite_max',
                                 'timeOnSite_mean'])


In [80]:
tf =tr

tp = tf.groupby('fullVisitorId').apply(my_agg)
tp


,timeOnSite_sum,timeOnSite_min,timeOnSite_max,timeOnSite_mean
fullVisitorId,,,,
0000245437374675368,91.0,91.0,91.0,91.0
0001191766179392657,0.0,NaN,NaN,NaN
0003631840334189025,64.0,64.0,64.0,64.0
0004374401845204055,260.0,260.0,260.0,260.0
0004803397127474847,2890.0,2890.0,2890.0,2890.0
...,...,...,...,...
9991882289486375263,112.0,112.0,112.0,112.0
9996018333110892344,98.0,98.0,98.0,98.0
9996276006553512804,20.0,20.0,20.0,20.0


In [84]:
tp

,timeOnSite_sum,timeOnSite_min,timeOnSite_max,timeOnSite_mean
fullVisitorId,,,,
0000245437374675368,91.0,91.0,91.0,91.0
0001191766179392657,0.0,NaN,NaN,NaN
0003631840334189025,64.0,64.0,64.0,64.0
0004374401845204055,260.0,260.0,260.0,260.0
0004803397127474847,2890.0,2890.0,2890.0,2890.0
...,...,...,...,...
9991882289486375263,112.0,112.0,112.0,112.0
9996018333110892344,98.0,98.0,98.0,98.0
9996276006553512804,20.0,20.0,20.0,20.0


In [85]:
tpp= tp.reset_index()


In [87]:
tpp[tpp['fullVisitorId']=='8934116514970143966']

,fullVisitorId,timeOnSite_sum,timeOnSite_min,timeOnSite_max,timeOnSite_mean
8203,8934116514970143966,86.0,15.0,43.0,28.666667


In [88]:
tr[tr['fullVisitorId']=='8934116514970143966']['totals_timeOnSite']

1       28.0
2868    15.0
7755    43.0
Name: totals_timeOnSite, dtype: float64

In [89]:
 tff = pd.merge(tr, tpp, left_on='fullVisitorId', right_on='fullVisitorId')

In [90]:
tff[tff['fullVisitorId']=='8934116514970143966'].T

,1,2,3
channelGrouping,Referral,Referral,Referral
date,2017-10-16 00:00:00,2017-10-16 00:00:00,2017-11-30 00:00:00
fullVisitorId,8934116514970143966,8934116514970143966,8934116514970143966
visitId,1508176307,1508160211,1512072970
visitNumber,6,5,7
visitStartTime,1508176307,1508160211,1512072970
device_browser,Chrome,Chrome,Chrome
device_browserVersion,NaN,NaN,NaN
device_browserSize,NaN,NaN,NaN
device_operatingSystem,Chrome OS,Chrome OS,Chrome OS


In [20]:
from datetime import datetime, timedelta
tr["date"] = pd.to_datetime(tr["date"], infer_datetime_format=True, format="%Y%m%d")
def getTimeFramewithFeatures(tr, k=1):

    tf = tr.loc[(tr['date'] >= min(tr['date']) + timedelta(days=168*(k-1))) 
              & (tr['date'] < min(tr['date']) + timedelta(days=168*k))]
    
    print(tf.shape)

    tf_fvid = set(tr.loc[(tr['date'] >= min(tr['date']) + timedelta(days=168*k + 46 )) 
                       & (tr['date'] < min(tr['date']) + timedelta(days=168*k + 46 + 62))]['fullVisitorId'])

    tf_returned = tf[tf['fullVisitorId'].isin(tf_fvid)]
    
    tf_tst = tr[tr['fullVisitorId'].isin(set(tf_returned['fullVisitorId']))
             & (tr['date'] >= min(tr['date']) + timedelta(days=168*k + 46))
             & (tr['date'] < min(tr['date']) + timedelta(days=168*k + 46 + 62))]
    
    tf_target = tf_tst.groupby('fullVisitorId')[['totals_totalTransactionRevenue']].sum().apply(np.log1p, axis=1).reset_index()
    tf_target['ret'] = 1
    tf_target.rename(columns={'totals_totalTransactionRevenue': 'target'}, inplace=True)
    
    tf_nonret = pd.DataFrame()
    tf_nonret['fullVisitorId'] = list(set(tf['fullVisitorId']) - tf_fvid)    
    tf_nonret['target'] = 0
    tf_nonret['ret'] = 0
    
    tf_target = pd.concat([tf_target, tf_nonret], axis=0).reset_index(drop=True)
    # len(set(tf['fullVisitorId'])), len(set(tf_target['fullVisitorId']))
    tf_maxdate = max(tf['date'])
    tf_mindate = min(tf['date'])

    tf = tf.groupby('fullVisitorId').agg({
            'geoNetwork_networkDomain': {'networkDomain': lambda x: x.dropna().max()},
            'geoNetwork_city': {'city': lambda x: x.dropna().max()},
            'device_operatingSystem': {'operatingSystem': lambda x: x.dropna().max()},
            'geoNetwork_metro': {'metro': lambda x: x.dropna().max()},
            'geoNetwork_region': {'region': lambda x: x.dropna().max()},
            'channelGrouping': {'channelGrouping': lambda x: x.dropna().max()},
            'trafficSource_referralPath': {'referralPath': lambda x: x.dropna().max()},
            'geoNetwork_country': {'country': lambda x: x.dropna().max()},
            'trafficSource_source': {'source': lambda x: x.dropna().max()},
            'trafficSource_medium': {'medium': lambda x: x.dropna().max()},
            'trafficSource_keyword': {'keyword': lambda x: x.dropna().max()},
            'device_browser':  {'browser': lambda x: x.dropna().max()},
            'trafficSource_adwordsClickInfo.gclId': {'gclId': lambda x: x.dropna().max()},
            'device_deviceCategory': {'deviceCategory': lambda x: x.dropna().max()},
            'geoNetwork_continent': {'continent': lambda x: x.dropna().max()},
            'totals_timeOnSite': {'timeOnSite_sum': lambda x: x.dropna().sum(),
                                  'timeOnSite_min': lambda x: x.dropna().min(), 
                                  'timeOnSite_max': lambda x: x.dropna().max(),
                                  'timeOnSite_mean': lambda x: x.dropna().mean()},
            'totals_pageviews': {'pageviews_sum': lambda x: x.dropna().sum(),
                                 'pageviews_min': lambda x: x.dropna().min(), 
                                 'pageviews_max': lambda x: x.dropna().max(),
                                 'pageviews_mean': lambda x: x.dropna().mean()},
            'totals_hits': {'hits_sum': lambda x: x.dropna().sum(), 
                            'hits_min': lambda x: x.dropna().min(), 
                            'hits_max': lambda x: x.dropna().max(), 
                            'hits_mean': lambda x: x.dropna().mean()},
            'visitStartTime': {'visitStartTime_counts': lambda x: x.dropna().count()},
            'totals_sessionQualityDim': {'sessionQualityDim': lambda x: x.dropna().max()},
            'trafficSource_isTrueDirect': {'isTrueDirect': lambda x: x.dropna().max()},
            'totals_newVisits': {'newVisits_max': lambda x: x.dropna().max()},
            'device_isMobile': {'isMobile': lambda x: x.dropna().max()},
            'visitNumber': {'visitNumber_max' : lambda x: x.dropna().max()}, 
            'totals_totalTransactionRevenue':  {'totalTransactionRevenue_sum':  lambda x:x.dropna().sum()},
            'totals_transactions' : {'transactions' : lambda x:x.dropna().sum()},
            'date': {'first_ses_from_the_period_start': lambda x: x.dropna().min() - tf_mindate,
                     'last_ses_from_the_period_end': lambda x: tf_maxdate - x.dropna().max(),
                     'interval_dates': lambda x: x.dropna().max() - x.dropna().min(),
                     'unqiue_date_num': lambda x: len(set(x.dropna())) },            
                    })

    tf.columns = tf.columns.droplevel()

    tf = pd.merge(tf, tf_target, left_on='fullVisitorId', right_on='fullVisitorId')
    return tf

In [21]:
###Getting parts of train-set
print('Get 1st train part')
tr1 = getTimeFramewithFeatures(tr, k=1)
tr1.to_pickle(PATH/'tr1_clean')

Get 1st train part
(3832, 56)


SpecificationError: nested renamer is not supported